In [2]:
from omegaconf import DictConfig, OmegaConf
from etnn.bindingnet.bindingnetcc import BindingNetCC
import utils
import os
import pandas as pd
from torch_geometric.loader import DataLoader
import torch
from etnn.combinatorial_data import CombinatorialComplexData
from preprocess.single_graph_processing import (
    process_ligand_sdf,
    process_protein_pdb_ligand_style,
    merge_ligand_and_protein,
)

In [4]:
data_root = "data/bindingnetcc"
base_graphs_path = os.path.join(data_root, "subset_20p_base_graphs/preprocessed")
rcut3_path = os.path.join(data_root, "subset_20p_no_supercell_crossconnect_rcut_3/processed/merged_lifted.pt")
no_cross_connect_path = os.path.join(data_root, "subset_20p_no_supercell_no_connect_cross_self/processed/merged_lifted.pt")

In [6]:
no_cross_graphs = torch.load(no_cross_connect_path, weights_only=False)

In [19]:
print(no_cross_graphs[0].keys())
print(no_cross_graphs[1].keys())

dict_keys(['pos', 'cells_0', 'slices_0', 'cells_1', 'slices_1', 'x_0', 'x_1', 'mem_0', 'mem_1', 'adj_0_0_1', 'y'])
dict_keys(['pos', 'cells_0', 'slices_0', 'cells_1', 'slices_1', 'x_0', 'x_1', 'mem_0', 'mem_1', 'adj_0_0_1', 'y'])


In [20]:
rcut3_path = torch.load(rcut3_path, weights_only=False)

In [21]:
print(len(rcut3_path))

3


In [22]:
rcut3_prelift_path = os.path.join(data_root, "subset_20p_no_supercell_crossconnect_rcut_3/preprocessed/merged")

In [24]:
rcut3_merged_count = len(os.listdir(rcut3_prelift_path))
print(rcut3_merged_count)

137515
